# Save products table to BigQuery

O primeiro passo para rodar este notebook é configurar suas chaves de acesso na GCP. Caso ainda não possua uma API key
consulte a documentação para gerar a API Key na forma de JSON, salve o arquivo em seu projeto e renomeie para
google-cloud-key.json.

Aqui também são definidas as variaveis de ambiente deste notebook usadas para a execução do arquivo *karhub_extraction.py*, e as variaveis utilizadas no método de persistencia dos dataframes no BigQuery

In [1063]:
#
# As variáveis de ambiente e globais abaixo precisam ser configuradas
# antes de rodar o notebook.
#

# Caminho para a chave do service account do GCP
# Veja: https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
%env GOOGLE_APPLICATION_CREDENTIALS=./google-cloud-key.json

# Credenciais para criancao de user
%env FULL_NAME=Marcela Barella
%env EMAIL=marcela.barella@hotmail.com

BQ_PROJECT='karhub-data-engineer-test'
BQ_DATASET='cadastro_produto'
BQ_TABLE="kh_data_engineer_teste_marcelabarella"

env: GOOGLE_APPLICATION_CREDENTIALS=./google-cloud-key.json
env: FULL_NAME=Marcela Barella
env: EMAIL=marcela.barella@hotmail.com


Esta celula ira instalar as dependencias necessarias para a execução deste notebook, incluindo a biblioteca de requests usada para a execução do arquivo que captura os dados vindos da API de veículos

In [1064]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

You should consider upgrading via the '/Users/grelinhodediamante/Projects/karhub/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/grelinhodediamante/Projects/karhub/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/grelinhodediamante/Projects/karhub/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/grelinhodediamante/Projects/karhub/venv/bin/python3 -m pip install --upgrade pip' command.


Aqui o arquivo de extração de veículos será executado utilizando as variaveis de ambiente do projeto.

In [1065]:
%run -i 'karhub_extraction.py'

In [1066]:
import json
import pandas as pd
import numpy as np

In [1067]:
alias_df = pd.read_csv('karhub-alias.csv')

In [1068]:
alias_df

,Fabricante,Código Fabricante,Alias
0,KarHub,FO0352,fo0352
1,KarHub,FO0353,fo0353
2,KarHub,FT0417,ft0417
3,KarHub,VW0127,vw0127
4,KarHub,PE0452,pe0452
...,...,...,...
1245,KarHub,I-VW0468,VW0468
1246,KarHub,I-CH0512,I-CH0512
1247,KarHub,I-CH0512,ich0512
1248,KarHub,I-MI0611,I-MI0611


Aqui é feita a verificação da existência de campos nulos no dataframe para verificar se é necessario algum tipo de normalização e não obstante se o numero de valores nulos "compensam" essa normalização em termos de recursos e tempo de execução.

In [1070]:
alias_df.isnull().sum().sum()

0

In [1071]:
autoparts_products_df = pd.read_excel('karhub_autoparts_1.xlsx')

In [1072]:
autoparts_products_df

,Nome SKU,Fabricante,Código,Composição,Categoria,Nome Atributo,Valor Atributo
0,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Largura (cm),18.0
1,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Altura (cm),9.0
2,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Comprimento (cm),8.0
3,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Bruto (Kg),0.600
4,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Líquido (Kg),0.600
...,...,...,...,...,...,...,...
3313,Mola de Suspensão Dianteira Helicoidal,KarHub,I-VW0406,UNITARY,Mola Helicoidal,Largura (cm),40
3314,Mola de Suspensão Dianteira Helicoidal,KarHub,I-VW0406,UNITARY,Mola Helicoidal,Altura (cm),25
3315,Mola de Suspensão Dianteira Helicoidal,KarHub,I-VW0406,UNITARY,Mola Helicoidal,Comprimento (cm),25
3316,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0605,UNITARY,Mola Helicoidal,Largura (cm),40


O mesmo é feito aqui, outro adendo é que esta soma também está sendo feita para verificar se na fonte de dados referencia há inconsistencias.

In [1073]:
autoparts_products_df.isnull().sum().sum()

0

Algumas linhas da coluna **Valor Atributo** possuem valores numericos, porém, com o tipo **String** e a separação das casas decimais é feita com *vírgula*, então foi feita uma substituíção do separador pelo separador *ponto*.

In [1074]:
autoparts_products_df['Valor Atributo'] = autoparts_products_df['Valor Atributo'].str.replace(',','.')

Removendo linhas duplicadas (idênticas):

In [1075]:
autoparts_products_df.drop_duplicates(inplace=True)

Aqui fazemos uma contagem para verificar se existe duplicidade de registros com o mesmo código mas valores diferentes para o mesmo atributo.

In [1076]:
counts = autoparts_products_df.groupby(['Código', 'Nome Atributo']).agg(pd.Series.nunique)

In [1077]:
counts[counts['Valor Atributo'] > 1]

,,Nome SKU,Fabricante,Composição,Categoria,Valor Atributo
Código,Nome Atributo,,,,,
I-FO2016,Posição,1,1,1,1,2
I-FO2017,Posição,1,1,1,1,2
I-FO2038,Posição,1,1,1,1,2
I-VW2000,Posição,1,1,1,1,2
I-VW2003,Posição,1,1,1,1,2
I-VW2008,Posição,1,1,1,1,2
I-VW2009,Posição,1,1,1,1,2
I-VW2010,Posição,1,1,1,1,2
I-VW2022,Posição,1,1,1,1,2


Nota se que o atributo **Posição** é multivalorado para algumas peças. Inspecionando os exemplos, nota se que algumas peças possuem posição tanto *traseira* como *dianteira*. Estas linhas serão agregadas em uma lista para facilitar a operação de *pivotar* da tabela.

In [1078]:
autoparts_products_df = autoparts_products_df.groupby(['Código', 'Nome Atributo']).agg({ 
    'Valor Atributo': list,
    'Nome SKU': 'first',
    'Fabricante': 'first',
    'Composição': 'first',
    'Categoria': 'first'
})

In [1079]:
autoparts_products_df.reset_index(inplace=True)

In [1080]:
autoparts_products_df['Valor Atributo'] = autoparts_products_df['Valor Atributo'].apply(lambda x: x[0] if len(x) == 1 else x)

In [1081]:
autoparts_products_df

,Código,Nome Atributo,Valor Atributo,Nome SKU,Fabricante,Composição,Categoria
0,0054,Altura (cm),9.0,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
1,0054,Comprimento (cm),8.0,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
2,0054,Largura (cm),18.0,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
3,0054,Peso Bruto (Kg),0.600,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
4,0054,Peso Líquido (Kg),0.600,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
...,...,...,...,...,...,...,...
3279,vw0524,Comprimento (cm),8.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3280,vw0524,Largura (cm),18.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3281,vw0524,Peso Bruto (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3282,vw0524,Peso Líquido (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal


In [1082]:
autoparts_products_df.dtypes

Código            object
Nome Atributo     object
Valor Atributo    object
Nome SKU          object
Fabricante        object
Composição        object
Categoria         object
dtype: object

Verifica se com um exemplo que os valores foram agregados corretamente:

In [1083]:
autoparts_products_df[autoparts_products_df['Código'] == 'I-VW2047']

,Código,Nome Atributo,Valor Atributo,Nome SKU,Fabricante,Composição,Categoria
2997,I-VW2047,Altura (cm),38,Kit de Molas de Suspensão Helicoidais Cangoorun,KarHub,UNITARY,Mola Helicoidal
2998,I-VW2047,Comprimento (cm),38,Kit de Molas de Suspensão Helicoidais Cangoorun,KarHub,UNITARY,Mola Helicoidal
2999,I-VW2047,Largura (cm),38,Kit de Molas de Suspensão Helicoidais Cangoorun,KarHub,UNITARY,Mola Helicoidal
3000,I-VW2047,Peso Bruto (Kg),13,Kit de Molas de Suspensão Helicoidais Cangoorun,KarHub,UNITARY,Mola Helicoidal
3001,I-VW2047,Peso Líquido (Kg),2.0,Kit de Molas de Suspensão Helicoidais Cangoorun,KarHub,UNITARY,Mola Helicoidal
3002,I-VW2047,Posição,"[Dianteira, Traseira]",Kit de Molas de Suspensão Helicoidais Cangoorun,KarHub,UNITARY,Mola Helicoidal


In [1084]:
autoparts_products_df

,Código,Nome Atributo,Valor Atributo,Nome SKU,Fabricante,Composição,Categoria
0,0054,Altura (cm),9.0,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
1,0054,Comprimento (cm),8.0,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
2,0054,Largura (cm),18.0,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
3,0054,Peso Bruto (Kg),0.600,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
4,0054,Peso Líquido (Kg),0.600,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
...,...,...,...,...,...,...,...
3279,vw0524,Comprimento (cm),8.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3280,vw0524,Largura (cm),18.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3281,vw0524,Peso Bruto (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3282,vw0524,Peso Líquido (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal


Após normalizar ambos os dataframes, é feito o "de para" onde os campos de código no dataframe *autoparts_products_df* são substituídos pelo valor contido na coluna **Código Fabricante** do dataframe *alias_df* descartando as linhas que não tem equivalencia (inner join).

In [1085]:
autoparts_products_df = autoparts_products_df.merge(alias_df[['Alias', 'Código Fabricante']], 
                                                    left_on='Código',
                                                    right_on='Alias', how='inner')

As colunas remanecentes são removidas e a coluna contendo os IDs é renomeada para facilitar o processo de junção com o dataframe de veículos.

In [1086]:
autoparts_products_df = autoparts_products_df.drop(['Código', 'Alias'], axis=1)

In [1087]:
autoparts_products_df = autoparts_products_df.rename(columns={'Código Fabricante': 'Código'})

In [1088]:
autoparts_products_df

,Nome Atributo,Valor Atributo,Nome SKU,Fabricante,Composição,Categoria,Código
0,Altura (cm),25,Trizeta,KarHub,UNITARY,Transmissão,I-VW0384
1,Comprimento (cm),25,Trizeta,KarHub,UNITARY,Transmissão,I-VW0384
2,Largura (cm),40,Trizeta,KarHub,UNITARY,Transmissão,I-VW0384
3,Número de Espirais,10.75,Trizeta,KarHub,UNITARY,Transmissão,I-VW0384
4,Peso Bruto (Kg),4.5,Trizeta,KarHub,UNITARY,Transmissão,I-VW0384
...,...,...,...,...,...,...,...
3261,Comprimento (cm),8.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VW0524
3262,Largura (cm),18.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VW0524
3263,Peso Bruto (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VW0524
3264,Peso Líquido (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VW0524


In [1089]:
pivoted_autoparts_products_df = autoparts_products_df.pivot(
    index='Código',
    columns='Nome Atributo',
    values='Valor Atributo',
)

In [1090]:
pivoted_autoparts_products_df.reset_index(inplace=True)

In [1091]:
pivoted_autoparts_products_df

Nome Atributo,Código,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição
0,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
1,FO0231,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
2,FO0300,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
3,FO0301,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
4,FO0306,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Traseira
...,...,...,...,...,...,...,...,...,...
472,VW0446,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Traseira
473,VW0474,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
474,VW0478,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
475,VW0524,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira


In [1092]:
pivoted_autoparts_products_df.dtypes

Nome Atributo
Código                object
Altura (cm)           object
Comprimento (cm)      object
Largura (cm)          object
Motor                 object
Número de Espirais    object
Peso Bruto (Kg)       object
Peso Líquido (Kg)     object
Posição               object
dtype: object

In [1093]:
pivoted_autoparts_products_df = pivoted_autoparts_products_df.explode('Posição')

In [1094]:
pivoted_autoparts_products_df[pivoted_autoparts_products_df['Código'] == 'I-VW2047']

Nome Atributo,Código,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição
447,I-VW2047,38,38,38,NaN,NaN,13,2.0,Dianteira
447,I-VW2047,38,38,38,NaN,NaN,13,2.0,Traseira


Após "pivotar" o dataframe *autoparts_products_df* ele será unido com o dataframe original contendo as demais colunas que não foram "pivotadas", novamente com **inner join**.

In [1095]:
pivoted_autoparts_products_df = pivoted_autoparts_products_df.merge(autoparts_products_df, on='Código', how='inner')

In [1096]:
pivoted_autoparts_products_df

,Código,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição,Nome Atributo,Valor Atributo,Nome SKU,Fabricante,Composição,Categoria
0,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Altura (cm),9.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
1,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Comprimento (cm),8.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
2,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Largura (cm),18.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Peso Bruto (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
4,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Peso Líquido (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Comprimento (cm),8.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3388,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Largura (cm),18.0,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3389,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Peso Bruto (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3390,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Peso Líquido (Kg),0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal


In [1097]:
pivoted_autoparts_products_df = pivoted_autoparts_products_df.groupby(['Código', 'Posição']).first().reset_index()

In [1098]:
pivoted_autoparts_products_df.drop(['Nome Atributo', 'Valor Atributo'], axis=1, inplace=True)

In [1099]:
pivoted_autoparts_products_df

,Código,Posição,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Nome SKU,Fabricante,Composição,Categoria
0,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
1,FO0231,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
2,FO0300,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3,FO0301,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
4,FO0306,Traseira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,VW0446,Traseira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal
494,VW0474,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Chicote eletrico,KarHub,UNITARY,Sonda Lambda
495,VW0478,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
496,VW0524,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal


In [1100]:
pivoted_autoparts_products_df.dtypes

Código                object
Posição               object
Altura (cm)           object
Comprimento (cm)      object
Largura (cm)          object
Motor                 object
Número de Espirais    object
Peso Bruto (Kg)       object
Peso Líquido (Kg)     object
Nome SKU              object
Fabricante            object
Composição            object
Categoria             object
dtype: object

In [1101]:
pivoted_autoparts_products_df[pivoted_autoparts_products_df['Código'] == 'I-VW2047']

,Código,Posição,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Nome SKU,Fabricante,Composição,Categoria
467,I-VW2047,Dianteira,38,38,38,None,None,13,2.0,Kit de Molas de Suspensão Helicoidais Cangoorun,KarHub,UNITARY,Mola Helicoidal
468,I-VW2047,Traseira,38,38,38,None,None,13,2.0,Kit de Molas de Suspensão Helicoidais Cangoorun,KarHub,UNITARY,Mola Helicoidal


In [1102]:
with open('./cars_data.json', 'r') as file:
    raw_data = json.load(file)

In [1103]:
df_cars = pd.DataFrame.from_dict(raw_data)

Aqui a soma de valores nulos no dataframe de veículos indica as colunas em que estão esses valores nulos. Vale ressaltar que não necessáriamente todos os valores nulos são de um mesmo veículo (linha), mas sim estão distribuídos de forma que alguns veículos possuem atributos nulos, como por exemplo, Marca e Modelo, enquanto outros podem conter outros atributos nulos como somente o complemento.

In [1104]:
df_cars.isnull().sum()

Fabricante             0
Código Fabricante      0
Marca                128
Modelo               128
Ano                  128
Complemento          128
dtype: int64

Dada a existencia de valores nulos é feita então a normalização dos campos onde campos com valores de tipo **NaN** são substituidos pelo tipo **None** para que quando inseridos no BigQuery sua visualização fique mais clara.

In [1105]:
df_cars.fillna(np.nan).replace([np.nan], [None])

,Fabricante,Código Fabricante,Marca,Modelo,Ano,Complemento
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P | 1.0 MPI 8V - 4P | 1.0 ...
1,KarHub,FO0352,FORD,FIESTA,1997.0,1.0 MPI 8V - 2P | 1.3 MPI 8V CLX 2P | 1.3 MP...
2,KarHub,FO0352,FORD,FIESTA,1998.0,1.0 MPI 8V CLASS 2P | 1.0 MPI 8V CLASS 4P | ...
3,KarHub,FO0353,FORD,KA,1997.0,1.0i 8V - 2P | 1.3 MPI 8V CLX 2P | 1.0 MPI 8...
4,KarHub,FO0353,FORD,KA,1998.0,1.0i 8V - 2P | 1.3 MPI 8V CLX 2P | 1.0 MPI 8...
...,...,...,...,...,...,...
4210,KarHub,I-VW0468,AUDI,A3,2004.0,1.8 TURBO 20V 180CV - 4P | 1.6 8V - 2P | 1.8...
4211,KarHub,I-VW0468,AUDI,A3,2005.0,1.6 8V - 4P | 1.8 20V - 4P | 1.8 20V - 2P |...
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 20V - 4P | 1.6 8V - 4P | 1.8 TURBO 20V 1...
4213,KarHub,I-CH0512,None,None,None,None


Para que a coluna **Ano** possa ser convertida para o tipo inteiro, uma vez que ano que o ano é um valor inteiro, as linhas não preenchidas foram preenchidas com o valor zero, por mais que não faça sentido um carro do ano 0, sendo assim 0 é equivalente a **null** neste caso.

In [1106]:
df_cars['Ano'] = df_cars['Ano'].fillna(0)
df_cars['Ano'] = df_cars['Ano'].astype(int)

In [1107]:
df_cars

,Fabricante,Código Fabricante,Marca,Modelo,Ano,Complemento
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V CLASS 2P | 1.0 MPI 8V - 4P | 1.0 ...
1,KarHub,FO0352,FORD,FIESTA,1997,1.0 MPI 8V - 2P | 1.3 MPI 8V CLX 2P | 1.3 MP...
2,KarHub,FO0352,FORD,FIESTA,1998,1.0 MPI 8V CLASS 2P | 1.0 MPI 8V CLASS 4P | ...
3,KarHub,FO0353,FORD,KA,1997,1.0i 8V - 2P | 1.3 MPI 8V CLX 2P | 1.0 MPI 8...
4,KarHub,FO0353,FORD,KA,1998,1.0i 8V - 2P | 1.3 MPI 8V CLX 2P | 1.0 MPI 8...
...,...,...,...,...,...,...
4210,KarHub,I-VW0468,AUDI,A3,2004,1.8 TURBO 20V 180CV - 4P | 1.6 8V - 2P | 1.8...
4211,KarHub,I-VW0468,AUDI,A3,2005,1.6 8V - 4P | 1.8 20V - 4P | 1.8 20V - 2P |...
4212,KarHub,I-VW0468,AUDI,A3,2006,1.8 20V - 4P | 1.6 8V - 4P | 1.8 TURBO 20V 1...
4213,KarHub,I-CH0512,None,None,0,None


Antes de "explodir" em linhas diferentes é necessário dividir pelo separador pipe o item da coluna **Complemento** (string) em uma lista de strings onde cara item da lista será assinalado a uma linha no dataframe.

In [1108]:
df_cars = df_cars.assign(Complemento=df_cars['Complemento'].str.split('|'))

In [1109]:
df_cars = df_cars.explode('Complemento')

In [1110]:
df_cars

,Fabricante,Código Fabricante,Marca,Modelo,Ano,Complemento
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V CLASS 2P
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V - 4P
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V - 2P
0,KarHub,FO0352,FORD,FIESTA,1996,1.3 MPI 8V CLX 4P
0,KarHub,FO0352,FORD,FIESTA,1996,1.0 MPI 8V CLASS 4P
...,...,...,...,...,...,...
4212,KarHub,I-VW0468,AUDI,A3,2006,1.8 TURBO 20V 150CV - 4P
4212,KarHub,I-VW0468,AUDI,A3,2006,1.8 20V - 2P
4212,KarHub,I-VW0468,AUDI,A3,2006,1.8 TURBO 20V 180CV - 4P
4213,KarHub,I-CH0512,None,None,0,None


In [1111]:
df_cars.dtypes

Fabricante           object
Código Fabricante    object
Marca                object
Modelo               object
Ano                   int64
Complemento          object
dtype: object

Depois de "explodir" a coluna **Complemento** o dataframe de veículos será unido ao dataframe de produtos para então chegar ao dataframe final que será persistido no BigQuery.
Aqui a junção é feita considerando os valores de **Código** e **Código Fabricante** onde linhas sem equivalencia serão descartadas. Além disso as colunas pertinentes do dataframe de carros serão mantidas.

In [1112]:
products_df = pivoted_autoparts_products_df.merge(df_cars[['Código Fabricante', 'Marca', 'Modelo', 'Ano', 'Complemento']], 
                                            left_on='Código',
                                            right_on='Código Fabricante', how='inner')

In [1113]:
products_df.dtypes

Código                object
Posição               object
Altura (cm)           object
Comprimento (cm)      object
Largura (cm)          object
Motor                 object
Número de Espirais    object
Peso Bruto (Kg)       object
Peso Líquido (Kg)     object
Nome SKU              object
Fabricante            object
Composição            object
Categoria             object
Código Fabricante     object
Marca                 object
Modelo                object
Ano                    int64
Complemento           object
dtype: object

In [1114]:
products_df

,Código,Posição,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Nome SKU,Fabricante,Composição,Categoria,Código Fabricante,Marca,Modelo,Ano,Complemento
0,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FO0191,FORD,BELINA,1970,1.4 8V - 2P
1,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FO0191,FORD,BELINA,1971,1.4 8V - 2P
2,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FO0191,FORD,BELINA,1972,1.4 8V - 2P
3,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FO0191,FORD,BELINA,1973,1.4 8V - 2P
4,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FO0191,FORD,BELINA,1974,1.4 8V - 2P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28805,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VW0525,VOLKSWAGEN,SAVEIRO,2012,1.6 MI 8V CS G.V 2P
28806,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VW0525,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V TROOPER CS G.V 2P
28807,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VW0525,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V CS G.V 2P
28808,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VW0525,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V CE G.V 2P


Dado que a coluna **Código Fabricante** pode ser considerada uma coluna duplicada, ela será removida.

In [1115]:
products_df.drop(['Código Fabricante'], axis=1, inplace=True)

Os nomes das colunas também foram normalizados para sua persistencia no BigQuery

In [1116]:
products_df.columns = products_df.columns.str.replace(' ', "_").str.replace('(', "").str.replace(')', "").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

In [1117]:
products_df.dtypes

codigo                object
posicao               object
altura_cm             object
comprimento_cm        object
largura_cm            object
motor                 object
numero_de_espirais    object
peso_bruto_kg         object
peso_liquido_kg       object
nome_sku              object
fabricante            object
composicao            object
categoria             object
marca                 object
modelo                object
ano                    int64
complemento           object
dtype: object

In [1118]:
products_df

,codigo,posicao,altura_cm,comprimento_cm,largura_cm,motor,numero_de_espirais,peso_bruto_kg,peso_liquido_kg,nome_sku,fabricante,composicao,categoria,marca,modelo,ano,complemento
0,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1970,1.4 8V - 2P
1,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1971,1.4 8V - 2P
2,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1972,1.4 8V - 2P
3,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1973,1.4 8V - 2P
4,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1974,1.4 8V - 2P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28805,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2012,1.6 MI 8V CS G.V 2P
28806,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V TROOPER CS G.V 2P
28807,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V CS G.V 2P
28808,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V CE G.V 2P


Terminadas as normalizações, limpezas e junções dos dataframes serão adicionadas duas colunas, uma contendo a data e hora atuais do momento em que executei o dataframe para persistencia no banco e outra contendo meu nome.

In [1119]:
products_df['candidate_name'] = 'Marcela Barella'
products_df['dt_insert'] = pd.Timestamp('now')

In [1120]:
products_df.dtypes

codigo                        object
posicao                       object
altura_cm                     object
comprimento_cm                object
largura_cm                    object
motor                         object
numero_de_espirais            object
peso_bruto_kg                 object
peso_liquido_kg               object
nome_sku                      object
fabricante                    object
composicao                    object
categoria                     object
marca                         object
modelo                        object
ano                            int64
complemento                   object
candidate_name                object
dt_insert             datetime64[ns]
dtype: object

In [1121]:
products_df

,codigo,posicao,altura_cm,comprimento_cm,largura_cm,motor,numero_de_espirais,peso_bruto_kg,peso_liquido_kg,nome_sku,fabricante,composicao,categoria,marca,modelo,ano,complemento,candidate_name,dt_insert
0,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1970,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
1,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1971,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
2,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1972,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
3,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1973,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
4,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1974,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28805,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2012,1.6 MI 8V CS G.V 2P,Marcela Barella,2023-05-04 00:38:41.211644
28806,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V TROOPER CS G.V 2P,Marcela Barella,2023-05-04 00:38:41.211644
28807,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V CS G.V 2P,Marcela Barella,2023-05-04 00:38:41.211644
28808,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V CE G.V 2P,Marcela Barella,2023-05-04 00:38:41.211644


In [1122]:
from google.cloud import bigquery

def save_to_bigquery(df):
    bq_table_id = f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}'

    table_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("codigo", 'STRING'),
            bigquery.SchemaField("posicao", 'STRING'),
            bigquery.SchemaField("altura_cm", 'FLOAT'),
            bigquery.SchemaField("comprimento_cm", 'FLOAT'),
            bigquery.SchemaField("largura_cm", 'FLOAT'),
            bigquery.SchemaField("motor", 'STRING'),
            bigquery.SchemaField("numero_de_espirais", 'FLOAT'),
            bigquery.SchemaField("peso_bruto_kg", 'FLOAT'),
            bigquery.SchemaField("peso_liquido_kg", 'FLOAT'),
            bigquery.SchemaField("nome_sku", 'STRING'),
            bigquery.SchemaField("fabricante", 'STRING'),
            bigquery.SchemaField("composicao", 'STRING'),
            bigquery.SchemaField("categoria", 'STRING'),
            bigquery.SchemaField("marca", 'STRING'),
            bigquery.SchemaField("modelo", 'STRING'),
            bigquery.SchemaField("ano", 'INTEGER'),
            bigquery.SchemaField("complemento", 'STRING'),
            bigquery.SchemaField("candidate_name", 'STRING'),
            bigquery.SchemaField("dt_insert", 'DATETIME')
        ],
        
        # Sobescrever a tabela inteira quando rodar o script
        write_disposition="WRITE_TRUNCATE",
    )

    client = bigquery.Client()
    client.load_table_from_dataframe(df, bq_table_id, job_config=table_config)

In [1123]:
products_df

,codigo,posicao,altura_cm,comprimento_cm,largura_cm,motor,numero_de_espirais,peso_bruto_kg,peso_liquido_kg,nome_sku,fabricante,composicao,categoria,marca,modelo,ano,complemento,candidate_name,dt_insert
0,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1970,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
1,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1971,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
2,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1972,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
3,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1973,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
4,FO0191,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,FORD,BELINA,1974,1.4 8V - 2P,Marcela Barella,2023-05-04 00:38:41.211644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28805,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2012,1.6 MI 8V CS G.V 2P,Marcela Barella,2023-05-04 00:38:41.211644
28806,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V TROOPER CS G.V 2P,Marcela Barella,2023-05-04 00:38:41.211644
28807,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V CS G.V 2P,Marcela Barella,2023-05-04 00:38:41.211644
28808,VW0525,Dianteira,9.0,8.0,18.0,None,None,0.600,0.600,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,VOLKSWAGEN,SAVEIRO,2013,1.6 MI 8V CE G.V 2P,Marcela Barella,2023-05-04 00:38:41.211644


In [1124]:
products_df['codigo'] = products_df['codigo'].astype(str)
products_df['posicao'] = products_df['posicao'].astype(str).replace('None', None)
products_df['altura_cm'] = products_df['altura_cm'].fillna('').replace('', np.nan).astype(float)
products_df['comprimento_cm'] = products_df['comprimento_cm'].fillna('').replace('', np.nan).astype(float)
products_df['largura_cm'] = products_df['largura_cm'].fillna('').replace('',np.nan).astype(float)
products_df['motor'] = products_df['motor'].astype(str).replace('None', None)
products_df['numero_de_espirais'] = products_df['numero_de_espirais'].fillna('').replace('',np.nan).astype(float)
products_df['peso_bruto_kg'] = products_df['peso_bruto_kg'].fillna('').replace('',np.nan).astype(float)
products_df['peso_liquido_kg'] = products_df['peso_liquido_kg'].fillna('').replace('',np.nan).astype(float)
products_df['nome_sku'] = products_df['nome_sku'].astype(str).replace('None', None)
products_df['fabricante'] = products_df['fabricante'].astype(str).replace('None', None)
products_df['composicao'] = products_df['composicao'].astype(str).replace('None', None)
products_df['categoria'] = products_df['categoria'].astype(str).replace('None', None)
products_df['marca'] = products_df['marca'].astype(str).replace('None', None)
products_df['modelo'] = products_df['modelo'].astype(str).replace('None', None)
products_df['complemento'] = products_df['complemento'].astype(str).replace('None', None)
products_df['candidate_name'] = products_df['candidate_name'].astype(str).replace('None', None)

In [1125]:
save_to_bigquery(products_df)